In [1]:
#Load the libraries
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from google.cloud import language_v2

import warnings
warnings.filterwarnings('ignore')

['IMDB Dataset.csv']


In [11]:
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "ml-spec-0488fd916a4d.json"

In [12]:
imdb_data=pd.read_csv('gs://engo-ml_spec2023-demo3/input/IMDB Dataset.csv')
print(imdb_data.shape)

(50000, 2)


In [13]:
imdb_data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [14]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,25000


In [15]:
imdb_data['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [26]:
imdb_data['sentiment_code'] =  [1 if x == "positive" else 0 for x in imdb_data["sentiment"]]

In [57]:
def sample_analyze_sentiment(text_content: str = "I am so happy and joyful.") -> None:
    """
    Analyzes Sentiment in a string.

    Args:
        text_content: The text content to analyze.
    """

    client = language_v2.LanguageServiceClient()

    # text_content = 'I am so happy and joyful.'

    # Available types: PLAIN_TEXT, HTML
    document_type_in_plain_text = language_v2.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language_code = "en"
    document = {
        "content": text_content,
        "type_": document_type_in_plain_text,
        "language_code": language_code,
    }

    # Available values: NONE, UTF8, UTF16, UTF32
    # See https://cloud.google.com/natural-language/docs/reference/rest/v2/EncodingType.
    encoding_type = language_v2.EncodingType.UTF8

    response = client.analyze_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )
    # Get overall sentiment of the input document
    #print(f"Document sentiment score: {response.document_sentiment.score}")
    #print(f"Document sentiment magnitude: {response.document_sentiment.magnitude}")
    # Get sentiment for all sentences in the document
    #for sentence in response.sentences:
        #print(f"Sentence text: {sentence.text.content}")
        #print(f"Sentence sentiment score: {sentence.sentiment.score}")
        #print(f"Sentence sentiment magnitude: {sentence.sentiment.magnitude}")

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    # print(f"Language of the text: {response.language_code}")
    return int(response.document_sentiment.score > 0), response.document_sentiment.score, response.document_sentiment.magnitude

In [144]:
scores = []
magnitudes = []
categories = []
for index, row in imdb_data.iloc[0:5].iterrows():
    category, score, magnitude = sample_analyze_sentiment(row["review"])
    categories.append(category)
    scores.append(score)
    magnitudes.append(magnitude)
    np.save('categories_example.npy', categories)
    np.save('scores_example.npy', scores)
    np.save('magnitudes_example.npy', magnitudes)

In [122]:
accuracy = accuracy_score(imdb_data['sentiment_code'], categories)

In [132]:
print("Cloud Natural Language Sentiment Analysis Accuracy on IMDB dataset:", str(int(accuracy*100)) + "%")

Cloud Natural Language Sentiment Analysis Accuracy on IMDB dataset: 89%


In [127]:
imdb_data["class_from_api"] = categories
imdb_data["scores_from_api"] = scores
imdb_data["magnitudes_from_api"] = magnitudes

In [134]:
imdb_data[imdb_data["sentiment_code"] != imdb_data["class_from_api"]]

,review,sentiment,sentiment_code,class_from_api,scores_from_api,magnitudes_from_api
0,One of the other reviewers has mentioned that ...,positive,1,0,-0.011,10.439000
10,Phil the Alien is one of those quirky films wh...,negative,0,1,0.199,3.972000
16,Some films just simply should not be remade. T...,positive,1,0,-0.082,6.791000
18,"I remember this film,it was the first film i h...",positive,1,0,-0.151,1.572000
25,The Karen Carpenter Story shows a little more ...,positive,1,0,-0.077,7.203000
...,...,...,...,...,...,...
49961,<br /><br />Headlines warn us of the current c...,positive,1,0,-0.426,7.401000
49977,"Dog Bite Dog isn't going to be for everyone, b...",positive,1,0,-0.096,9.789000
49991,"Les Visiteurs, the first movie about the medie...",negative,0,1,0.012,10.920000
49992,John Garfield plays a Marine who is blinded by...,positive,1,0,-0.290,5.266000


In [139]:
imdb_data["scores_from_api"].describe()

count    50000.000000
mean        -0.000936
std          0.464706
min         -0.955000
25%         -0.385000
50%         -0.035000
75%          0.380000
max          0.957000
Name: scores_from_api, dtype: float64

In [141]:
imdb_data["class_from_api"].value_counts()

0    26084
1    23916
Name: class_from_api, dtype: int64

In [138]:
imdb_data["magnitudes_from_api"].describe()

count    50000.000000
mean         9.315149
std          5.888617
min          0.185000
25%          5.538000
50%          7.815000
75%         11.339000
max         76.549004
Name: magnitudes_from_api, dtype: float64